# Quantized (Ultra Lite) TF Model

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sys import getsizeof
import os
import h5py


In [2]:
tf.__version__


'2.5.0'

In [3]:
# Loading the Keras model
KERAS_MODEL_NAME = '../models/3_sl_aug_model'
model = keras.models.load_model(KERAS_MODEL_NAME)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 16)        64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0

In [4]:
# to get model size
def get_folder_size(folder_path: str):
    size = 0
    for path, dirs, files in os.walk(folder_path):
        for file in files:
            fp = os.path.join(path, file)
            size += os.path.getsize(fp)
    return size


In [5]:
print(f'Keras Model Size: {get_folder_size(KERAS_MODEL_NAME)} Bytes')


Keras Model Size: 4877596 Bytes


In [6]:
# creating test set to calculate model accuracy
valid_df = pd.read_csv("../data/asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_test = valid_df['label']
del valid_df['label']

# Separate out our image vectors
x_test = valid_df.values

y_test_numpy = y_test.values
y_test = keras.utils.to_categorical(y_test, 24)

# Normalize our image data
x_test = x_test / 255

x_test = x_test.reshape(-1, 28, 28, 1)


In [7]:
loss, acc = model.evaluate(x_test, y_test)
print(f"loss: {loss}, Accuracy: {acc}")


225/225 [==============================] - 2s 7ms/step - loss: 0.0025 - accuracy: 0.9989
loss: 0.002462963340803981, Accuracy: 0.9988845586776733


In [8]:
TF_QUANT_MODEL_NAME = '../models/tf_quant_model.tflite'


In [9]:
# tfLite convertor
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tf_lite_model = tf_lite_converter.convert()


INFO:tensorflow:Assets written to: C:\Users\KUMAR~1.SHA\AppData\Local\Temp\tmp5c6orfd2\assets


In [10]:
with open(TF_QUANT_MODEL_NAME, "wb") as f:
    f.write(tf_lite_model)


In [11]:
print(f"Model Size: {os.path.getsize(TF_QUANT_MODEL_NAME)} Bytes")


Model Size: 571808 Bytes


In [12]:
os.path.getsize(TF_QUANT_MODEL_NAME) / get_folder_size(KERAS_MODEL_NAME)


0.11723152142981912

## Accuracy of Quant Model

In [13]:
interpreter = tf.lite.Interpreter(model_path=TF_QUANT_MODEL_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(f"Input Shape: {input_details[0]['shape']}")
print(f"Input Type: {input_details[0]['dtype']}")
print(f"Output Shape: {output_details[0]['shape']}")
print(f"Output Type: {output_details[0]['dtype']}")


Input Shape: [ 1 28 28  1]
Input Type: <class 'numpy.float32'>
Output Shape: [ 1 24]
Output Type: <class 'numpy.float32'>


### Resize tensor shape

In [14]:
interpreter.resize_tensor_input(input_details[0]['index'], x_test.shape)
interpreter.resize_tensor_input(output_details[0]['index'], y_test.shape)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(f"Input Shape: {input_details[0]['shape']}")
print(f"Input Type: {input_details[0]['dtype']}")
print(f"Output Shape: {output_details[0]['shape']}")
print(f"Output Type: {output_details[0]['dtype']}")


Input Shape: [7172   28   28    1]
Input Type: <class 'numpy.float32'>
Output Shape: [7172   24]
Output Type: <class 'numpy.float32'>


In [15]:
x_test.dtype


dtype('float64')

In [16]:
x_test_numpy = np.array(x_test, dtype=np.float32)


### Predictions

In [17]:
interpreter.set_tensor(input_details[0]['index'], x_test_numpy)
interpreter.invoke()

quant_predictions = interpreter.get_tensor(output_details[0]['index'])
print('Prediction result shape: ', quant_predictions.shape)
prediction_classes = np.argmax(quant_predictions, axis=1)


Prediction result shape:  (7172, 24)


In [18]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(prediction_classes, y_test_numpy)
print('Test Accuracy of Quantized model: ', acc)


Test Accuracy of Quantized model:  0.9993028443948689
